In [ ]:
from scipy.stats import norm, genextreme

In [ ]:
import numpy as np
import pandas as pd

from astropy.table import Table
from astropy import units
from astropy.units import imperial
from astropy.visualization import quantity_support

import ipywidgets as ipw

from climate_explore import ClimateData, TempData

quantity_support()

%matplotlib widget

from matplotlib import pyplot as plt

In [ ]:
all_data = Table.read("psci378-extremes.csv")
ge = None

In [ ]:
c = ClimateData(all_data)

In [ ]:
pad_me = ipw.Layout(padding="5px")
locations = ipw.Dropdown(options=c.locations, default=None)
extreme_types_drop = ipw.ToggleButtons(
    options=[
        ("High", "TMAX"),
        ("Low", "TMIN"),
        #("Max precipitation", "PRCP")
    ]
)
extreme_types_label = ipw.Label("Extreme type")
extreme_types = ipw.VBox(children=[extreme_types_label, extreme_types_drop])
unit_selector = ipw.ToggleButtons(description="Units", options=[("metric", units.Celsius), ("english", units.imperial.Fahrenheit)], layout=pad_me) # ipw.Dropdown(options=["metric", "english"])

center = ipw.VBox()
left = ipw.VBox()
left.children = [extreme_types, unit_selector]

#plot_selector = ipw.Tab()

app = ipw.AppLayout(center=center, left_sidebar=left, header=locations, pane_heights=[1,10,0])

#locations.observe(update_location,

In [ ]:
def get_extreme_type_name():
    return extreme_types_drop.options[extreme_types_drop.index][0]

In [ ]:
with plt.ioff():
    fig, ax = plt.subplots()
    lines = ax.plot([1800, 2025], [50, 50])
    xmin, xmax = ax.get_xlim()
    ymin, ymax = ax.get_ylim()
    low_year = ax.plot([xmin, xmin], [ymin, ymax], "-", alpha=0.7, color="green")
    high_year = ax.plot([xmax, xmax], [ymin, ymax], "-", alpha=0.7, color="green")
    trend = ax.plot([1800, 2025], [50, 50], alpha=0, color="orange")
    trend = trend[0]
    ax.set_xlabel("Year")
    ax.set_ylabel(f"AVERAGE {get_extreme_type_name()} Temperature")
    ax.grid()


In [ ]:
with plt.ioff():
     pe_fig, pe_ax = plt.subplots()

In [ ]:
with plt.ioff():
    ret_fig, ret_ax = plt.subplots()
    ret_ax.text(.2, .5, "Do tab 2 first")

In [ ]:
year_range = ipw.IntRangeSlider(min=c.years.min(), max=c.years.max(), step=1, description="Select years", layout=dict(width="90%"))
fig.canvas.layout = ipw.Layout(width="90%")
year_range.value = (year_range.min, year_range.max)
tabs = ipw.Tab()

range_and_average = ipw.VBox()

range_and_average.children = [year_range, fig.canvas]

pe_fig.canvas.layout = ipw.Layout(width="90%")
hot_temp =ipw.IntSlider(value=100, min=50, max=130, description="Hot temp")
offset_center_by = ipw.IntText(value=3, description="Temp offset")
probabilities = ipw.HTML("")

offset_and_highs = ipw.VBox()
offset_and_highs.children = [hot_temp, offset_center_by, probabilities, pe_fig.canvas]

ret_fig.canvas.layout = ipw.Layout(width="90%")
return_values = ipw.Box(children=[ret_fig.canvas])

tabs.children = [range_and_average, offset_and_highs, return_values]
tabs.titles = ("1) Select years", "2) Annual highs", "3) Return values")
center.children = [tabs]

In [ ]:
selected_location_data = ""

In [ ]:
def update_trend_line():
    bobber = selected_location_data
    trend_fit = bobber.fit_trend(group_by="YEAR")
    trend.set_data(bobber.data['YEAR'], trend_fit(bobber.data['YEAR']))
    return bobber

In [ ]:
def make_subtitle():
    return ""

In [ ]:
def update_graph_line(dummy=None):
    selected_location_data.start_year, selected_location_data.end_year = year_range.value
    selected_location_data.extreme_type = get_extreme_type_name().lower()
    selected_location_data.display_unit = unit_selector.value
    years, temps = selected_location_data.max_min_temps_by_year(group_by='YEAR')
    lines[0].set_data(years, temps)
    ax.set_ylim((np.nanmin(temps) - 10 * temps.unit), (np.nanmax(temps) + 10 * temps.unit))
    ax.set_xlim(years.min() - 5, years.max() + 5)
    ax.set_ylabel(f"AVERAGE {get_extreme_type_name()} Temperature ({unit_selector.value:latex})")
    subset = update_trend_line()
    rate_change = subset.fit_trend(group_by="YEAR").coef[1] * subset.display_unit * 100
    rate_change_string = rate_change.to_string(format="latex", precision=3)
    ax.set_title(f"{c.location} \nchange in 100 years {rate_change_string}")
    update_year_markers({})
    make_extrema_plots({})
    make_return_value_plots({})

In [ ]:
def update_location(change):
    global selected_location_data
    c.set_location(locations.value)
    selected_location_data = TempData(data=c.selected_data,
                                      display_unit=unit_selector.value,
                                      extreme_type=get_extreme_type_name().lower()
                                     )

    ax.set_title(f"{c.location}")
    update_graph_line()
    year_range.min, year_range.max = selected_location_data.year_range

    year_range.value = (year_range.min, year_range.max)
    trend.set_alpha(1)
    fig.canvas.draw()
    fig.canvas.flush_events()

In [ ]:
def update_year_markers(change):
    xmin, xmax = year_range.value
    ymin, ymax = ax.get_ylim()
    high_year[0].set_data([xmax, xmax], [ymin, ymax])
    low_year[0].set_data([xmin, xmin], [ymin, ymax])
    selected_location_data.start_year, selected_location_data.end_year = year_range.value
    subset = update_trend_line()
    rate_change = subset.fit_trend(group_by="YEAR").coef[1] * subset.display_unit * 100
    ax.set_title(f"{c.location} \nchange in 100Y {rate_change:.3f}")
    fig.canvas.draw()
    fig.canvas.flush_events()


In [ ]:
def make_extrema_plots(_):
    global selected_location_data
    global loc_offset
    global ge, ge_counter, ge_future
    global tempa, yr
    
    yr, tempa = selected_location_data.max_min_temps_by_year(group_by="YEAR", aggregate_by=np.nanmax)
    pe_ax.clear()
    sigma, loc, shape = genextreme.fit(tempa, loc=tempa.value.mean(), scale=tempa.value.std())
    loc_offset = offset_center_by.value
    ge = genextreme(sigma, loc, shape)
    ge_counter = genextreme(sigma,loc - loc_offset, shape)
    ge_future = genextreme(sigma,loc + loc_offset, shape)
    
    pe_ax.hist(tempa, density=True, alpha=0.5)
    #pe_ax = model.extremes.hist(density=True)
    xmin, xmax = pe_ax.get_xlim()
    x = np.linspace(xmin - 5, xmax + 5, num=200)
    mod = pe_ax.plot(x, ge.pdf(x))[0]
    mod_color = mod.get_color()
    hot = hot_temp.value #105 # model.extremes.max() - 5
    pe_ax.plot(x, ge.pdf(x))
    pe_ax.fill_between(x, ge.pdf(x), where=(x >= hot), color="green", alpha=0.5)
    pe_ax.plot(x, ge_counter.pdf(x), color="black", alpha=0.5)
    #pe_ax.fill_between(x, ge_counter.pdf(x),  where=(x >= hot), color="black", alpha=0.3)
    pe_ax.plot(x, ge_future.pdf(x), color="red", alpha=0.5)
    #pe_ax.fill_between(x, ge_future.pdf(x),  where=(x >= hot), color="red", alpha=0.3)
    pe_ax.set_title(f"{c.location}")
    pe_ax.set_xlabel(f"{selected_location_data.extreme_type.upper()} temperature, {selected_location_data.display_unit}")
    pe_ax.grid()
    probs = [
        f"Current world: {1 - ge.cdf(hot):.3f}",
        f"No climate change {1 - ge_counter.cdf(hot):.3f}",
        f"Hot future {1 - ge_future.cdf(hot):.3f}"
    ]
    probabilities.value = " ".join(probs)
    pe_fig.canvas.draw()
    pe_fig.canvas.flush_events()

In [ ]:
def return_period_data(data):
    to_sort = data.copy()
    to_sort.sort()
    n_d = len(to_sort)
    i = np.arange(1, n_d + 1)
    probty = (n_d - i + 1) / (n_d + 1)
    return 1 / probty, to_sort
    
def make_return_value_plots(_):
    """
    Make the return value plots
    """
    global selected_location_data
    global loc_offset
    global ge, ge_counter, ge_future
    yr, tempa = selected_location_data.max_min_temps_by_year(group_by="YEAR", aggregate_by=np.nanmax)
    ret_ax.clear()
    return_period = np.logspace(0, 5, num=1000)
    if ge is None:
        ret_ax.text(.2, .5, "Do tab 2 first")

    ret = ge.isf(1/return_period)
    ret_counter = ge_counter.isf(1/return_period)
    ret_future = ge_future.isf(1/return_period)
    ret_ax.plot(return_period, ret, color="green", label="actual world")
    ret_ax.plot(return_period, ret_counter, color="black", label="no warming")
    ret_ax.plot(return_period, ret_future, color="red", label="future")
    ret_ax.set_xlabel("Return period (years)")
    ret_ax.set_ylabel(f"Temperature ({unit_selector.value:latex})")
    ret_ax.set_ylim()
    ret_ax.grid()
    ret_ax.semilogx()

In [ ]:
def update_temp_unit(change):
    update_graph_line()

In [ ]:
locations.observe(update_location, names="value")

In [ ]:
year_range.observe(update_year_markers, names="value")
year_range.observe(make_extrema_plots, names="value")
year_range.observe(make_return_value_plots, names="value")

unit_selector.observe(update_temp_unit, names="value")
extreme_types_drop.observe(update_graph_line, names="value")
hot_temp.observe(make_extrema_plots, names="value")
offset_center_by.observe(make_extrema_plots, names="value")
hot_temp.observe(make_return_value_plots, names="value")
offset_center_by.observe(make_return_value_plots, names="value")


In [ ]:
locations.value = "FARGO HECTOR INTERNATIONAL AIRPORT, ND US"

In [ ]:
app